In [ ]:
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
base_dir = Path('./dump_match_pairs_nordland')

In [ ]:
match_dict = {}
for f in tqdm(sorted(base_dir.iterdir())):
    if not f.is_file():
        continue
    im_num1 = int(f.stem[7:12])
    im_num2 = int(f.stem[20:25])
    if im_num1 > 451 or im_num2 > 451:
        continue
    matchfile = np.load(f)
    matches = matchfile['matches']
    confidences = matchfile['match_confidence']
    pos_matches_indices = matches>-1
    pos_matches = matches[pos_matches_indices]
    num_matches = len(pos_matches)
    weighted_matches = np.ones(num_matches) * confidences[pos_matches_indices]
    weighted_matches_sum = weighted_matches.sum()
    # print(weighted_matches)
    if im_num1 not in match_dict:
        match_dict[im_num1] = {}
    match_dict[im_num1][im_num2] = weighted_matches_sum

In [ ]:
n_values = [1, 2, 5]
best_matches_diffs = []
gt_images = np.array(list(match_dict.keys()))
q_images = gt_images.copy()
correct_at_n = np.zeros(len(n_values))
for idx1, im_num1 in enumerate(match_dict):
    all_matches = np.array(list(match_dict[im_num1].values()))
    # print(len(all_matches))
    match_sorted_idx = np.argsort(all_matches)[::-1]
    for i, n in enumerate(n_values):
        # print(gt_images[idx1], q_images[match_sorted_idx][0])
        if np.any(np.in1d(q_images[match_sorted_idx][:n], gt_images[idx1])):
        # if np.any(np.in1d(match_sorted_idx[:n], idx1)):
            # print('test', n)
            correct_at_n[i:] += 1
            break
recall_at_n = correct_at_n / len(match_dict)
all_recalls = {}  # make dict for output
for i, n in enumerate(n_values):
    all_recalls[n] = recall_at_n[i]
all_recalls